In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
  raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [ ]:
from agents import Agent, Runner

agent = Agent(
  name="Basic Agent",
  instructions="You are a helpful assistant. Respond on in all caps.",
  model="o4-mini"
)

result = await Runner.run(agent, "Hello, how are you?")
result.final_output

In [ ]:
joke_agent = Agent(
  name="Joke Agent",
  instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
  model="o4-mini"
)

topic = "Cats"
result = await Runner.run(joke_agent, topic)
result.final_output

In [ ]:
language_agent = Agent(
  name="Language Agent",
  instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
  model="o4-mini"
)

joke_result = await Runner.run(joke_agent, topic)
transleted_result = await Runner.run(language_agent, f"Translate this joke to Icelandic: {joke_result.final_output} ")
print(f"Original Joke: {joke_result.final_output}")
print(f"Translated Joke: {transleted_result.final_output}")

## Structured Outputs

Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require a specific formatting or data extraction.

In [ ]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
  title: str
  ingredients: list[str]
  instructions: str
  cooking_time: int # in minutes
  servings: int

recipe_agent = Agent(
  name="Recipe Agent",
  instructions="You are an agent for creating recipes. You will be given the name of a food and your job is to output that as an actual detailed recipe. The cooking time should be in minutes.",
  output_type=Recipe,
  model="o4-mini"
)

response = await Runner.run(recipe_agent, "Chocolate Cake")
response.final_output

## Tool Calling

Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require real-time data or specific actions.

In [ ]:
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city.
    """
    # This is a placeholder implementation. In a real application, you would call a weather API.
    print(f"Fetching weather for {city}...")
    return f"The current weather in {city} is sunny."

@function_tool
def get_temperature(city: str) -> str:
    """
    Get the current temperature for a given city.
    """
    # This is a placeholder implementation. In a real application, you would call a weather API.
    print(f"Fetching temperature for {city}...")
    return f"The current temperature in {city} is 20°C."

weather_agent = Agent(
  name="Weather Agent",
  instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, you should respond with 'I can only provide weather information.'",
  model="o4-mini",
  tools=[get_weather, get_temperature]
)
result = await Runner.run(weather_agent, "What is the weather in Reykjavik?")
result.final_output

In [ ]:
from agents import Agent, Runner, WebSearchTool

news_agent = Agent(
  name="News Reporter",
  instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
  tools=[WebSearchTool()],
  model="o4-mini"
)

result = await Runner.run(news_agent, "Find me the latest news on the president. Only display any positive news articles. Keep the output line length to a maximum of 80 characters by using line break. If there are no positive articles, just say 'No positive news found.'")
print(f"Latest News about the president: \n\n{result.final_output}")